In [1]:
import cv2
import os
import numpy as np
import time
import scipy
import tensorflow as tf

from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import Adam,SGD,RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split


In [2]:
dataPath = 'data_color/'
picture_size = 96
batch_size  = 64
var_seed = 17
no_of_classes = 8
epochs = 75

In [3]:
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, validation_split=0.2)

train_set = datagen_train.flow_from_directory(#dataPath+"train",
                                            dataPath,
                                            subset="training",
                                            target_size = (picture_size,picture_size),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle = True, 
                                            seed  = var_seed)

val_set = datagen_train.flow_from_directory(#dataPath+"test",
                                            dataPath,
                                            subset="validation",
                                            target_size = (picture_size,picture_size),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle = False, 
                                            seed  = var_seed)

Found 23237 images belonging to 8 classes.
Found 5805 images belonging to 8 classes.


## vgg16 color model

In [8]:
vgg16 = tf.keras.applications.VGG16(input_shape=(picture_size, picture_size, 3), include_top=False, weights='imagenet')
vgg16.trainable = False

vgg16_model = tf.keras.Sequential([
    vgg16,

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(no_of_classes, activation='softmax')
])

opt = Adam(learning_rate= 0.0001)
vgg16_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               2359808   
                                                                 
 batch_normalization_7 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 activation_7 (Activation)   (None, 512)               0         
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                      

In [9]:
checkpoint = ModelCheckpoint("models/color/vgg16_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0.0001,
                          patience=6,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=6,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

vgg16_model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [10]:
%%time
history = vgg16_model.fit(train_set,
                                epochs=epochs,
                                validation_data = val_set,
                                callbacks=callbacks_list
                                )

Epoch 1/75
364/364 [==============================] - ETA: 0s - loss: 1.6397 - accuracy: 0.4077
Epoch 1: val_accuracy improved from -inf to 0.39862, saving model to models/color\vgg16_model.h5
364/364 [==============================] - 347s 951ms/step - loss: 1.6397 - accuracy: 0.4077 - val_loss: 1.9936 - val_accuracy: 0.3986 - lr: 0.0010
Epoch 2/75


e:\Estudios\MCDD\Mod_12_DeepLearning\TrabajoFinal\Proyecto\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


364/364 [==============================] - ETA: 0s - loss: 1.2449 - accuracy: 0.5389
Epoch 2: val_accuracy improved from 0.39862 to 0.41809, saving model to models/color\vgg16_model.h5
364/364 [==============================] - 332s 913ms/step - loss: 1.2449 - accuracy: 0.5389 - val_loss: 2.1081 - val_accuracy: 0.4181 - lr: 0.0010
Epoch 3/75
364/364 [==============================] - ETA: 0s - loss: 1.0970 - accuracy: 0.5943
Epoch 3: val_accuracy did not improve from 0.41809
364/364 [==============================] - 334s 918ms/step - loss: 1.0970 - accuracy: 0.5943 - val_loss: 1.8139 - val_accuracy: 0.3811 - lr: 0.0010
Epoch 4/75
364/364 [==============================] - ETA: 0s - loss: 0.9724 - accuracy: 0.6448
Epoch 4: val_accuracy improved from 0.41809 to 0.43376, saving model to models/color\vgg16_model.h5
364/364 [==============================] - 333s 914ms/step - loss: 0.9724 - accuracy: 0.6448 - val_loss: 1.8820 - val_accuracy: 0.4338 - lr: 0.0010
Epoch 5/75
364/364 [========

## Resnet Model

In [4]:
resnet = tf.keras.applications.ResNet50(input_shape=(picture_size, picture_size, 3), include_top=False, weights='imagenet')
resnet.trainable = False

resnet_model = tf.keras.Sequential([
    resnet,

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    

    tf.keras.layers.Dense(no_of_classes, activation='softmax')
])

opt = Adam(learning_rate= 0.0001)
resnet_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 3, 3, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 256)               4718848   
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                        

: 

In [12]:
checkpoint = ModelCheckpoint("models/color/resnet_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0.0001,
                          patience=6,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=6,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

resnet_model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [13]:
%%time
history = resnet_model.fit(train_set,
                                epochs=epochs,
                                validation_data = val_set,
                                callbacks=callbacks_list
                                )

Epoch 1/75
364/364 [==============================] - ETA: 0s - loss: 1.6282 - accuracy: 0.4019
Epoch 1: val_accuracy improved from -inf to 0.42825, saving model to models/color\resnet_model.h5
364/364 [==============================] - 327s 897ms/step - loss: 1.6282 - accuracy: 0.4019 - val_loss: 1.7271 - val_accuracy: 0.4283 - lr: 0.0010
Epoch 2/75
364/364 [==============================] - ETA: 0s - loss: 1.2788 - accuracy: 0.5243
Epoch 2: val_accuracy did not improve from 0.42825
364/364 [==============================] - 327s 898ms/step - loss: 1.2788 - accuracy: 0.5243 - val_loss: 1.7655 - val_accuracy: 0.4167 - lr: 0.0010
Epoch 3/75
364/364 [==============================] - ETA: 0s - loss: 1.1419 - accuracy: 0.5751
Epoch 3: val_accuracy improved from 0.42825 to 0.42963, saving model to models/color\resnet_model.h5
364/364 [==============================] - 328s 901ms/step - loss: 1.1419 - accuracy: 0.5751 - val_loss: 1.8244 - val_accuracy: 0.4296 - lr: 0.0010
Epoch 4/75
364/364

## Own Model

In [14]:
own_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),padding = 'same',input_shape = (96,96,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),   
    tf.keras.layers.Dropout(0.25),   
    
    
    tf.keras.layers.Conv2D(128,(3,3),padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),   
    tf.keras.layers.Dropout(0.25),    

    tf.keras.layers.Conv2D(128,(3,3),padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),   
    tf.keras.layers.Dropout(0.25),       

    tf.keras.layers.Conv2D(256,(3,3),padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),   
    tf.keras.layers.Dropout(0.30),  
    
    tf.keras.layers.Conv2D(256,(5,5),padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),   
    tf.keras.layers.Dropout(0.30),     
    
    tf.keras.layers.Conv2D(512,(5,5),padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),   
    tf.keras.layers.Dropout(0.30), 
       
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(no_of_classes, activation='softmax')
])

opt = Adam(learning_rate= 0.0001)
own_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
own_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 96, 96, 64)        1792      
                                                                 
 batch_normalization_9 (Bat  (None, 96, 96, 64)        256       
 chNormalization)                                                
                                                                 
 activation_9 (Activation)   (None, 96, 96, 64)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 48, 48, 64)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 48, 48, 128)      

In [15]:
checkpoint = ModelCheckpoint("models/color/own_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0.001,
                          patience=6,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=6,
                              verbose=1,
                              min_delta=0.001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]


own_model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [16]:
%%time
history = own_model.fit(train_set,
                                epochs=epochs,s
                                validation_data = val_set,
                                callbacks=callbacks_list
                                )

Epoch 1/75
364/364 [==============================] - ETA: 0s - loss: 1.6295 - accuracy: 0.3515
Epoch 1: val_accuracy improved from -inf to 0.35211, saving model to models/color\own_model.h5
364/364 [==============================] - 492s 1s/step - loss: 1.6295 - accuracy: 0.3515 - val_loss: 2.1930 - val_accuracy: 0.3521 - lr: 0.0010
Epoch 2/75
364/364 [==============================] - ETA: 0s - loss: 1.1991 - accuracy: 0.5177
Epoch 2: val_accuracy improved from 0.35211 to 0.49078, saving model to models/color\own_model.h5
364/364 [==============================] - 491s 1s/step - loss: 1.1991 - accuracy: 0.5177 - val_loss: 2.3147 - val_accuracy: 0.4908 - lr: 0.0010
Epoch 3/75
364/364 [==============================] - ETA: 0s - loss: 1.0794 - accuracy: 0.5703
Epoch 3: val_accuracy improved from 0.49078 to 0.52730, saving model to models/color\own_model.h5
364/364 [==============================] - 492s 1s/step - loss: 1.0794 - accuracy: 0.5703 - val_loss: 2.3338 - val_accuracy: 0.5273